<a href="https://colab.research.google.com/github/roshanrane24/QXI_DataScience_Roshan/blob/master/Unit7_AI_CNN/pytorch_MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.functional as F
import torch.optim
import numpy as np
import matplotlib.pyplot as plt

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [4]:
indices = np.arange(60000)
np.random.shuffle(indices)

trainset = torchvision.datasets.MNIST(root = './', train = True, download = True, transform=transf)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=False, sampler=torch.utils.data.RandomSampler(indices[:50000]))
valloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=False, sampler=torch.utils.data.RandomSampler(indices[50000:]))


testset = torchvision.datasets.MNIST(root = './', train = False, download = True, transform=transf)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


In [0]:
class CNN_MNIST(nn.Module):
    def __init__(self, in_ch, cls):
        super().__init__()
        self.Convolution1 = nn.Sequential(
            nn.Conv2d(in_ch, 500, 3),
            nn.MaxPool2d(2),
            nn.ReLU()
            )
        
        self.Convolution2 = nn.Sequential(
            nn.Conv2d(500,250,2),
            nn.BatchNorm2d(250),
            nn.ReLU()
        )

        self.Dropout25 = nn.Dropout(0.25)

        self.Dropout50 = nn.Dropout(0.5)

        self.Decode = nn.Sequential(
            nn.Linear(12*12*250, 100),
            nn.ReLU(),
            nn.Linear(100,50),
            nn.Sigmoid(),
            nn.Linear(50,cls)
        )

    def forward(self,x):
        x = self.Convolution1(x)
        x = self.Dropout25(x)
        x = self.Convolution2(x)
        x = self.Dropout50(x)

        x = x.view(x.size(0),12*12*250)

        x = self.Decode(x)
        return x

In [6]:
device

device(type='cuda', index=0)

In [0]:
model = CNN_MNIST(1, 10).to(device)

In [8]:
model.train()

CNN_MNIST(
  (Convolution1): Sequential(
    (0): Conv2d(1, 500, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (Convolution2): Sequential(
    (0): Conv2d(500, 250, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (Dropout25): Dropout(p=0.25, inplace=False)
  (Dropout50): Dropout(p=0.5, inplace=False)
  (Decode): Sequential(
    (0): Linear(in_features=36000, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [0]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
epochs = 10
correct, total = 0, 0
pred = []

In [10]:
for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch+1, epochs))
    for i, data in enumerate(trainloader):
        inputs, label = data
        inputs, label = inputs.to(device), label.to(device) 
        # inputs = inputs.view(-1,28*28)

        optimizer.zero_grad()

        output = model(inputs)

        _, pred = torch.max(output, 1)
        correct += (pred == label).sum().item()
        total += label.size(0)
        acc = (100 * correct / total)
        loss = criterion(output, label)

        loss.backward()
        optimizer.step()
    print('{} loss: {}                 accuracy: {}%'.format(i,round(float(loss),3),round(float(acc), 2)))

Epoch 1/10
1562 loss: 0.245                 accuracy: 92.57%
Epoch 2/10
1562 loss: 0.065                 accuracy: 95.4%
Epoch 3/10
1562 loss: 0.035                 accuracy: 96.5%
Epoch 4/10
1562 loss: 0.204                 accuracy: 97.12%
Epoch 5/10
1562 loss: 0.069                 accuracy: 97.53%
Epoch 6/10
1562 loss: 0.017                 accuracy: 97.84%
Epoch 7/10
1562 loss: 0.004                 accuracy: 98.07%
Epoch 8/10
1562 loss: 0.003                 accuracy: 98.24%
Epoch 9/10
1562 loss: 0.012                 accuracy: 98.4%
Epoch 10/10
1562 loss: 0.016                 accuracy: 98.52%


In [11]:
model.eval()

CNN_MNIST(
  (Convolution1): Sequential(
    (0): Conv2d(1, 500, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (Convolution2): Sequential(
    (0): Conv2d(500, 250, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (Dropout25): Dropout(p=0.25, inplace=False)
  (Dropout50): Dropout(p=0.5, inplace=False)
  (Decode): Sequential(
    (0): Linear(in_features=36000, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [0]:
predicted = []
cor, tot = 0,0

In [13]:
for i, data in enumerate(testloader):
        inputs, label = data
        inputs, label = inputs.to(device), label.to(device)

        output = model(inputs)

        _, pred = torch.max(output, 1)
        cor += (pred == label).sum().item()
        tot += label.size(0)

print('Accurcy of the model is {}%'.format((100 * cor/tot)))

Accurcy of the model is 99.14%
